# Hiphils with Dagesh in First Radical

First we import the necessary TF stuff

In [1]:
from tf.fabric import Fabric
TF = Fabric(locations='../text-fabric-data', modules='hebrew/etcbc4c')
api = TF.load('''
	sp nu gn ps vt vs g_word_utf8 pfm g_pfm_utf8 lex_utf8 g_vbs_utf8
''')
api.makeAvailableIn(globals())

This is Text-Fabric 2.3.2
Api reference : https://github.com/ETCBC/text-fabric/wiki/Api
Tutorial      : https://github.com/ETCBC/text-fabric/blob/master/docs/tutorial.ipynb
Data sources  : https://github.com/ETCBC/text-fabric-data
Data docs     : https://etcbc.github.io/text-fabric-data
Shebanq docs  : https://shebanq.ancient-data.org/text
Slack team    : https://shebanq.slack.com/signup
Questions? Ask shebanq@ancient-data.org for an invite to Slack
114 features found and 0 ignored
  0.00s loading features ...
   |     0.20s B g_word_utf8          from /home/jcuenod/Programming/text-fabric-data/hebrew/etcbc4c
   |     0.22s B lex_utf8             from /home/jcuenod/Programming/text-fabric-data/hebrew/etcbc4c
   |     0.12s B sp                   from /home/jcuenod/Programming/text-fabric-data/hebrew/etcbc4c
   |     0.11s B nu                   from /home/jcuenod/Programming/text-fabric-data/hebrew/etcbc4c
   |     0.10s B gn                   from /home/jcuenod/Programming/text-fabric

Compare preformatives to verbal stems

In [2]:
has_both = []
for w in F.otype.s('word'):
    preformative = F.g_pfm_utf8.v(w)
    verbal_stem = F.g_vbs_utf8.v(w)
    if (preformative is not "" and verbal_stem is not ""):
        has_both.append({
            "node": w,
            "word": F.g_word_utf8.v(w),
            "stem": F.vs.v(w),
            "pfm": preformative,
            "vbs": verbal_stem
        })
print("Number of words with both vbs and pfm in OT:", len(has_both))

Number of words with both vbs and pfm in OT: 937


Find all the hiphils

In [29]:
import re
def remove_non_root_consonants(word, pfm, vbs):
    # Will also remove all the accents/vowels until the first root cons
    prefix = pfm + vbs
    prefixWOCons = re.sub(r'[\u05D0-\u05EA\uFB2A\uFB2B\uFB4B]', "", prefix)
    number_of_consonants_in_prefix = len(prefix) - len(prefixWOCons)
    return_word = word
    for i in range(number_of_consonants_in_prefix):
        # This regex matches any accent/vowelish characters after a consonantish character
        return_word = re.sub(r'^[\u05D0-\u05EA\uFB2A\uFB2B\uFB4B][\u0590-\u05CF]*', "", return_word)
    return return_word
 
words = []
for w in F.otype.s('word'):
    if (F.vs.v(w) == "hif"):
        counter += 1
        line = F.g_word_utf8.v(w)
        preformative = F.g_pfm_utf8.v(w)
        verbal_stem = F.g_vbs_utf8.v(w)
        trimmed_line = remove_non_root_consonants(line, preformative, verbal_stem)
        words.append({
            "node": w,
            "word": line,
            "trim": trimmed_line,
            "vbs": verbal_stem,
            "pfm": preformative,
            "lex": F.lex_utf8.v(w)
        })
print("Number of words:", len(words))

Number of words: 9407


Now we do some checks to verify that our data makes sense

In [35]:
has_dagesh = []
for w in words:
    # Everything between the first two consonants
    first_two_cons = re.match(r'^[\u05D0-\u05EA\uFB2A\uFB2B\uFB4B]([\u0590-\u05CF]*)[\u05D0-\u05EA\uFB2A\uFB2B\uFB4B]', w["trim"])
    
    # if we don't have two consonants, just match the original string
    # (it just so happens that this doesn't change the result)
    if not first_two_cons:
        matched_chars = w["trim"]
    else:
        matched_chars = first_two_cons.group(1)
    
    # Now see if we have a dagesh...
    if re.search(r'\u05BC', matched_chars):
        has_dagesh.append(w)

print("Has dagesh in 1st root consonant:", len(has_dagesh))


Has dagesh in 1st root consonant: 1606


Now we just format things beautifully and create results

In [36]:
for w in has_dagesh:
    print("{:<12} {}".format( "{}".format(T.sectionFromNode(w["node"])), w["word"] ))
    print("\t", T.text( L.d(L.u(w["node"], otype="verse")[0], otype="word") ))

('Genesis', 2, 15) יַּנִּחֵ֣הוּ
	 וַיִּקַּ֛ח יְהוָ֥ה אֱלֹהִ֖ים אֶת־הָֽאָדָ֑ם וַיַּנִּחֵ֣הוּ בְגַן־עֵ֔דֶן לְעָבְדָ֖הּ וּלְשָׁמְרָֽהּ׃ 
('Genesis', 2, 21) יַּפֵּל֩
	 וַיַּפֵּל֩ יְהוָ֨ה אֱלֹהִ֧ים׀ תַּרְדֵּמָ֛ה עַל־הָאָדָ֖ם וַיִּישָׁ֑ן וַיִּקַּ֗ח אַחַת֙ מִצַּלְעֹתָ֔יו וַיִּסְגֹּ֥ר בָּשָׂ֖ר תַּחְתֶּֽנָּה׃ 
('Genesis', 3, 11) הִגִּ֣יד
	 וַיֹּ֕אמֶר מִ֚י הִגִּ֣יד לְךָ֔ כִּ֥י עֵירֹ֖ם אָ֑תָּה הֲמִן־הָעֵ֗ץ אֲשֶׁ֧ר צִוִּיתִ֛יךָ לְבִלְתִּ֥י אֲכָל־מִמֶּ֖נּוּ אָכָֽלְתָּ׃ 
('Genesis', 3, 13) הִשִּׁיאַ֖נִי
	 וַיֹּ֨אמֶר יְהוָ֧ה אֱלֹהִ֛ים לָאִשָּׁ֖ה מַה־זֹּ֣את עָשִׂ֑ית וַתֹּ֨אמֶר֙ הָֽאִשָּׁ֔ה הַנָּחָ֥שׁ הִשִּׁיאַ֖נִי וָאֹכֵֽל׃ 
('Genesis', 4, 15) הַכֹּות
	 וַיֹּ֧אמֶר לֹ֣ו יְהוָ֗ה לָכֵן֙ כָּל־הֹרֵ֣ג קַ֔יִן שִׁבְעָתַ֖יִם יֻקָּ֑ם וַיָּ֨שֶׂם יְהוָ֤ה לְקַ֨יִן֙ אֹ֔ות לְבִלְתִּ֥י הַכֹּות־אֹתֹ֖ו כָּל־מֹצְאֹֽו׃ 
('Genesis', 8, 21) הַכֹּ֥ות
	 וַיָּ֣רַח יְהוָה֮ אֶת־רֵ֣יחַ הַנִּיחֹחַ֒ וַיֹּ֨אמֶר יְהוָ֜ה אֶל־לִבֹּ֗ו לֹֽא־אֹ֠סִף לְקַלֵּ֨ל עֹ֤וד אֶת־הָֽאֲדָמָה֙ בַּעֲב֣וּר הָֽאָדָ֔ם כִּ֠י יֵ֣צֶר לֵ֧ב הָאָדָ֛ם רַ֖ע מִנְּ